In [1]:
# mount my Google Drive to save the notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# install libraries
!pip install -q diffusers transformers accelerate peft bitsandbytes kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.1 MB/s eta 0:00:00


In [5]:
# Kaggle API
from google.colab import files
files.upload() # upload kaggle.json API key
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# download Tom and Jerry dataset
!kaggle datasets download -d balabaskar/tom-and-jerry-image-classification -p /content/tom_and_jerry

Dataset URL: https://www.kaggle.com/datasets/balabaskar/tom-and-jerry-image-classification
License(s): CC0-1.0
 97% 424M/435M [00:01<00:00, 319MB/s]
100% 435M/435M [00:01<00:00, 396MB/s]


In [4]:
# unzip the dataset
!unzip -q /content/tom_and_jerry/tom-and-jerry-image-classification.zip -d /content/tom_and_jerry_dataset

In [6]:
# inspect dataset to verify and start data preparation
import os
dataset_path = '/content/tom_and_jerry_dataset/tom_and_jerry/tom_and_jerry'
print(os.listdir(dataset_path))

['tom_jerry_1', 'tom', 'tom_jerry_0', 'jerry']


In [9]:
# get all images from the dataset
import glob
from PIL import Image

image_paths = []
style_data_root = '/content/tom_and_jerry_dataset/tom_and_jerry/tom_and_jerry/'

# collect images from all subfolders -> goes through all(files and folders) in the style_data_root directory
for folder_name in os.listdir(style_data_root):
  folder_path = os.path.join(style_data_root, folder_name)
  if os.path.isdir(folder_path):
    # add all files to image_paths list
    image_paths.extend(glob.glob(os.path.join(folder_path, '*.jpg')))
    image_paths.extend(glob.glob(os.path.join(folder_path, '*.png'))) # just in case but I think its only .jpg files
print(f"Found  {len(image_paths)} images to use for training.")

Found  5478 images to use for training.


In [11]:
# load the base model = Stable Diffusion v1.5
import torch
from diffusers import StableDiffusionPipeline
from peft import LoraConfig, get_peft_model, PeftModel

model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda"

# load pipeline with 4-bit quantization
pipeline = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16, # try full 32bits precision later -> or rent a better GPU
    load_in_4bit=True, # model's weights and biases are loaded and stored using only 4 bits per value
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
pipeline.to(device)

pipeline.text_encoder.to(dtype=torch.float16, device=device)
pipeline.vae.to(dtype=torch.float16, device=device)

print("Base model loaded.")

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Keyword arguments {'load_in_4bit': True, 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_compute_dtype': torch.bfloat16} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Base model loaded.


In [12]:
# configure LoRA for UNet
from peft import LoraConfig, get_peft_model

# configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32, # often 2*r
    target_modules=["to_q", "to_k", "to_v", "to_out.0", "proj_in", "proj_out"],
    lora_dropout=0.05,
    bias="none",
)

# add LoRA adapters to the UNet -> making the UNet trainable, not the whole pipeline
unet = pipeline.unet
unet_lora = get_peft_model(unet, lora_config)
# verify LoRA application
unet_lora.print_trainable_parameters()

trainable params: 3,987,456 || all params: 863,508,420 || trainable%: 0.4618
